<a href="https://colab.research.google.com/github/Jahan08/Ambertools-CP2K-MM-QM-Biomolecular-Simulation/blob/main/Retrieve-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 935.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0


In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [47]:
target = new_client.target
target_query = target.search('KM-20L2')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,KM-20L2,34.0,False,CHEMBL614791,[],CELL-LINE,9606
1,[],Homo sapiens,KM-H2,16.0,False,CHEMBL2366250,[],CELL-LINE,9606
2,[],Homo sapiens,ST-KM-1,14.0,False,CHEMBL613972,[],CELL-LINE,9606
3,[],Homo sapiens,Cytosolic purine 5'-nucleotidase,12.0,False,CHEMBL3708197,"[{'accession': 'P49902', 'component_descriptio...",SINGLE PROTEIN,9606
4,"[{'xref_id': 'Q16881', 'xref_name': None, 'xre...",Homo sapiens,Thioredoxin reductase 1,10.0,False,CHEMBL1927,"[{'accession': 'Q16881', 'component_descriptio...",SINGLE PROTEIN,9606
5,[],Homo sapiens,Thioredoxin reductase,7.0,False,CHEMBL2096978,"[{'accession': 'Q16881', 'component_descriptio...",PROTEIN FAMILY,9606
6,[],Homo sapiens,KMS-11,7.0,False,CHEMBL4296454,[],CELL-LINE,9606
7,[],Homo sapiens,KMS-12-PE,6.0,False,CHEMBL2366257,[],CELL-LINE,9606
8,[],Homo sapiens,KMS-12-BM,6.0,False,CHEMBL4296455,[],CELL-LINE,9606


In [48]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL614791'

In [51]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="GI50")

In [53]:
df = pd.DataFrame.from_dict(res)
df

KeyboardInterrupt: ignored

In [43]:

# considering nM unit for MIC
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
len(df)

178

In [ ]:
# considering nM unit for MIC
df = df[(df['standard_units'] == '%')]
df = df[(df['relation'] == '=')]
len(df)

In [44]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i < 1000:
        STATUS.append("Yes") #active
        
    elif 20000 <= i <= 100000:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df

<ipython-input-44-d453deb383ef>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('value', 1)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,Activity
0,None,161577,[],CHEMBL692041,Compound was evaluated for in vitro activity a...,F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,IC50,nM,UO_0000065,None,0.90,Yes
1,None,162826,[],CHEMBL692041,Compound was evaluated for in vitro activity a...,F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,IC50,nM,UO_0000065,None,110.00,Yes
2,None,167754,[],CHEMBL692041,Compound was evaluated for in vitro activity a...,F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,IC50,nM,UO_0000065,None,1.30,Yes
3,None,667636,[],CHEMBL692030,In vitro cytotoxicity was determined against H...,F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,IC50,uM,UO_0000065,None,2300.00,IN
8,None,896464,[],CHEMBL686894,"Cytotoxicity against NCI tumor panel, HCC_2998...",F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,IC50,nM,UO_0000065,None,4.03,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,None,22850586,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4723415,Antiproliferative activity against human HCC 2...,F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,pIC50,None,UO_0000065,None,16982.44,IN
191,None,22850587,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4723415,Antiproliferative activity against human HCC 2...,F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,pIC50,None,UO_0000065,None,9549.93,IN
192,None,23256850,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4832256,Antiproliferative activity against human HCC 2...,F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,IC50,uM,UO_0000065,None,1780.00,IN
193,None,23256851,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4832256,Antiproliferative activity against human HCC 2...,F,None,None,BAO_0000190,BAO_0000219,...,HCC 2998,9606,None,None,IC50,uM,UO_0000065,None,3160.00,IN


In [ ]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 80:
        STATUS.append("Yes") #active
        
    elif i == 0:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

In [45]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

87

In [46]:
df.to_csv('bioactivity_data_HCC 2998_IC50_nM.csv', index=False)   